# Validação cruzada e aleatoriedade inicial (Aula 1)

In [ ]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x = dados[["preco", "idade_do_modelo","km_por_ano"]] # separar os dados antes dos dados para treino e pra teste é chamado de "holdout" (seguramos uma parte dos dados)
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))


Treinaremos com 7500 elementos e testaremos com 2500 elementos


In [ ]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acurácia do dummy stratified foi %.2f%%" % acuracia) #essa vai ser nussa baseline

A acurácia do dummy stratified foi 58.00%


In [ ]:
from sklearn.tree import DecisionTreeClassifier

SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 71.92%


In [ ]:
#mudaremos a seed, e vemos que mudou 4%. Precisamos saber como trabalhar com essa diferença de %. Vamos usar o cross validate
x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 15802 #seed diferente
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)
# vemos que por mudar a seed a acuracia muda. Isso significa que precisamos usar o cross validade que faz várias acuracias e tira uma média

Treinaremos com 7500 elementos e testaremos com 2500 elementos
A acurácia foi 75.12%


In [ ]:
from sklearn.model_selection import cross_validate #ele roda com seeds diferentes e tira uma média.(k-fold, corta em k partes)
#dessa forma diminuimos o impacto da aleatoridade
SEED = 301
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)#copia o modelo ali em cima
results = cross_validate(modelo, x, y, cv = 3, return_train_score=False)#quantos pedaços? 3
media = results['test_score'].mean() #media
desvio_padrao = results['test_score'].std()#desvio padrao
print("Accuracy com cross validation, 3 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao) * 100))# +- 2 desvios padrões -> 95%.
#se mudar o seed agora, não altera a acuracia. Mas se mudar o cv, muda (o ideal é escolher entre 5 e 10)

Accuracy com cross validation, 3 = [74.99, 76.57]


In [ ]:
SEED = 301#testando com cv 10
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 10, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 10 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 10 = [74.24, 77.32]


In [ ]:
SEED = 301#testando com cv 5
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 5, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 5 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 5 = [75.21, 76.35]


# KFold e aleatoriedade ( aula 2 )

In [ ]:
def imprime_resultados(results): #função que ele cria pra mostrar o resultado
  media = results['test_score'].mean()
  desvio_padrao = results['test_score'].std()
  print("Accuracy médio: %.2f" % (media * 100))
  print("Accuracy intervalo: [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao) * 100))

In [ ]:
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.78
Accuracy intervalo: [74.37, 77.19]


In [ ]:
SEED = 301 #Vamos usar dessa forma
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle = True)#dessa forma ele embaralha aleatóriamente antes de cross validar
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
imprime_resultados(results)#o resultado muda

Accuracy médio: 75.76
Accuracy intervalo: [73.26, 78.26]


# Simular situação horrível de azar ( aula 3 )

Pode ser "azar" como pode ser uma proporção de exemplos desbalanceado entre as classes.

In [ ]:
#quando a gente usava o splitfy, o teste ia proporcinalmente. Mas com o kfolds não valo mais aquilo
dados_azar = dados.sort_values("vendido", ascending=True)# e faz com que tenha a chance de escolher um 
x_azar = dados_azar[["preco", "idade_do_modelo","km_por_ano"]]#conjunto para teste horrível
y_azar = dados_azar["vendido"]
dados_azar.head()#estamos aqui simulando um casa 

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


In [ ]:
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)#usando sem o shufle

Accuracy médio: 57.84
Accuracy intervalo: [34.29, 81.39]


In [ ]:
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)#com o shufle

Accuracy médio: 75.78
Accuracy intervalo: [72.30, 79.26]


In [ ]:
from sklearn.model_selection import StratifiedKFold #agora ele stratifica mantendo as proporções
#vamos usar mais ele. Para saber o melhor caso precisa olhar nos guias do scikit learn
SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 75.78
Accuracy intervalo: [72.94, 78.62]


# Dados agrupáveis ( aula 4 )

In [ ]:
np.random.seed(SEED)#vamos tentar criar uma coluna "modelo", que vamos usar para usar o modeli para dados agrupábeis
dados['modelo'] = dados.idade_do_modelo + np.random.randint(-2, 3, size=10000)#queremos gerar 10000 números
dados.modelo = dados.modelo + abs(dados.modelo.min()) + 1 #soma esse 1 para tirar o valor negativo
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5


In [ ]:
dados.modelo.unique()

array([18, 24, 14,  6,  5, 13, 20, 19, 15,  2, 17, 12, 11, 16,  3,  7, 21,
       23, 10,  9, 22,  8,  4,  1])

In [ ]:
dados.modelo.value_counts()

20    901
19    798
18    771
21    723
17    709
16    668
14    621
22    575
15    573
13    557
12    511
11    401
10    371
23    370
9     336
8     278
7     206
24    199
6     181
5     108
4      76
3      44
2      17
1       6
Name: modelo, dtype: int64

Testando validação cruzada com GroupKFold

In [ ]:
from sklearn.model_selection import GroupKFold #vamos usar essa função para esses dados "agrupados" (itens que aparececem mais vezes)

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score=False)#precisamos falar qual é a coluna que vamos agrupar
imprime_resultados(results)

Accuracy médio: 75.78
Accuracy intervalo: [73.67, 77.90]


# Pipeline de treino e validação ( aula 5 )

In [ ]:
from sklearn.preprocessing import StandardScaler #alguns algoritmos são sensíveis a escala. O SVC é. O decision tree não
from sklearn.svm import SVC

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler() #fazendo a normalização.
scaler.fit(treino_x)
treino_x_escalado = scaler.transform(treino_x)
teste_x_escalado = scaler.transform(teste_x)

modelo = SVC()
modelo.fit(treino_x_escalado, treino_y)# x escalado
previsoes = modelo.predict(teste_x_escalado)# x escalado

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)
#Vemos que conseguimos uma acurácia boa. Mas agora vamos rodar o cross validate.

A acurácia foi 74.40%


In [ ]:
from sklearn.model_selection import GroupKFold #usando o cross validate com o SVC sem estar escalado. O resultado foi horrível. 
#Isso ocorre pq 
SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = SVC()
results = cross_validate(modelo, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score=False)
imprime_resultados(results) #conseguimos um resultado horrível. Pois não usamos os dados escalados.

Accuracy médio: 58.00
Accuracy intervalo: [56.10, 59.89]


In [ ]:
#preprocessamento
scaler = StandardScaler()
scaler.fit(x_azar)
x_azar_escalado = scaler.transform(x_azar) #escalando o x_azar, para testar o cross validate com os dados escalados.

In [ ]:
from sklearn.model_selection import GroupKFold #normalizando

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = SVC()
results = cross_validate(modelo, x_azar_escalado, y_azar, cv = cv, groups = dados.modelo, return_train_score=False)#usando o x_azar_escalado.
imprime_resultados(results)# após escalar os dados azar, vemos que o resultado ficou bem melhor

Accuracy médio: 76.71
Accuracy intervalo: [74.30, 79.12]


In [ ]:
from sklearn.pipeline import Pipeline #pipeline é um processo. Tem dois passos o scaler e o modelo
#dessa forma ele scala todas as vezes
SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao',scaler), ('estimador',modelo)])

cv = GroupKFold(n_splits = 10)
results = cross_validate(pipeline, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score=False)#vamos crossvalidar o pipeline inteiro
imprime_resultados(results)

Accuracy médio: 76.68
Accuracy intervalo: [74.28, 79.08]


# Estudo Pokemon (até aula 3 )

## Imports pretratamento

In [ ]:
import pandas as pd

In [ ]:
dados = pd.read_csv('pokemon.csv').drop(columns=["#",], axis=1)

In [ ]:
dados.drop(columns =['Defense', 'Sp. Def'],axis=1, inplace= True)

In [ ]:
dados

,Total,HP,Attack,Sp. Atk,Speed,Legendary
0,318,45,49,65,45,False
1,405,60,62,80,60,False
2,525,80,82,100,80,False
3,309,39,52,60,65,False
4,405,58,64,80,80,False
...,...,...,...,...,...,...
746,600,108,100,81,95,True
747,600,50,100,100,50,True
748,600,80,110,150,70,True
749,680,80,160,170,80,True


In [ ]:
troca ={True : 1,
        False : 0}
dados['Legendary'] = dados['Legendary'].map(troca)

In [ ]:
dados[:3]

,Total,HP,Attack,Sp. Atk,Speed,Legendary
0,318,45,49,65,45,0
1,405,60,62,80,60,0
2,525,80,82,100,80,0


## Treinando e verificando a diferença de acurácia

Hold out (separando treino e teste)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x=dados[['Total','HP','Attack','Sp. Atk','Speed']]
y=dados['Legendary']

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 563 elementos e testaremos com 188 elementos


Baseline (Dummy)

In [ ]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acurácia do dummy stratified foi %.2f%%" % acuracia) 

A acurácia do dummy stratified foi 92.02%


Treinando com o DecisionTreeClassifier e verificando a diferença de acurrácia para os diferentes SEEDS

In [ ]:
from sklearn.tree import DecisionTreeClassifier

SEED = 1
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 97.87%


In [ ]:
x=dados[['Total','HP','Attack','Sp. Atk','Speed']]
y=dados['Legendary']

SEED = 2
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)


Treinaremos com 563 elementos e testaremos com 188 elementos
A acurácia foi 98.94%


## Fazendo a validação cruzada

### cross_validate

In [ ]:
from sklearn.model_selection import cross_validate 

SEED = 301
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 5, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 5 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 3 = [94.49, 100.71]


### k-fold

In [ ]:
def imprime_resultados(results): #função que ele cria pra mostrar o resultado
  media = results['test_score'].mean()
  desvio_padrao = results['test_score'].std()
  print("Accuracy médio: %.2f" % (media * 100))
  print("Accuracy intervalo: [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao) * 100))

Sem shuffle

In [ ]:
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 97.47
Accuracy intervalo: [92.78, 102.15]


Com shuffle

In [ ]:
SEED = 301 
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 97.74
Accuracy intervalo: [94.20, 101.29]


### Azar

Criando os dados "azar

In [ ]:
dados_azar = dados.sort_values("Legendary", ascending=True)
x_azar = dados_azar[["Total", "HP","Attack","Sp. Atk","Speed"]]
y_azar = dados_azar["Legendary"]
dados_azar.head()

,Total,HP,Attack,Sp. Atk,Speed,Legendary
0,318,45,49,65,45,0
459,490,83,106,86,85,0
460,454,74,100,90,46,0
461,330,49,49,49,66,0
462,460,69,69,69,91,0


Testando sem Shuffle

In [ ]:
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 89.87
Accuracy intervalo: [44.02, 135.73]


Testando com Shuffle

In [ ]:
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10,shuffle = True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 97.73
Accuracy intervalo: [94.15, 101.32]


Usando o StratifiedKFold

In [ ]:
from sklearn.model_selection import StratifiedKFold 

SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio: 97.74
Accuracy intervalo: [94.35, 101.12]
